<a href="https://colab.research.google.com/github/Kessandra1/AER850_Project3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 0: INTRODUCTION

# Install Ultralytics (for YOLOv11), OpenCV, Matplotlib, numpy, and Pillow
!pip install -q ultralytics --upgrade
!pip install -q opencv-python-headless
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pillow

# Import
import os
import zipfile
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# For reproducibility
np.random.seed(42)

# Extract Project 3 Data zip folder
zip_path = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data.zip'
extract_path = '/content/project3_data'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
# STEP 1: OBJECT MASKING

# Load Motherboard JPEG image
img_path = os.path.join(extract_path, 'Project 3 Data', 'motherboard_image.JPEG')
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert image to grayscale for easier thresholding
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Perform thresholding with OpenCV
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# EDGE DETECTION using Contour detector (Canny) & Contour detection
edges = cv2.Canny(gray, 125, 225)
edges_dilated = cv2.dilate(edges, np.ones((5,5), np.uint8), iterations=2)
contours = cv2.findContours(edges_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
plt.imshow(edges_dilated, cmap='gray')
plt.title("Edges Detected")
plt.axis("off")
plt.show()

#Create a mask
largest_contour = max(contours, key=cv2.contourArea)
mask = np.zeros_like(gray)
cv2.drawContours(mask, [largest_contour], -1, 255, -1)
plt.imshow(mask, cmap='gray')
plt.title("Mask Image")
plt.axis("off")
plt.show()

#Extract PCB using cv2.bitwise_and() operator
pcb_extracted = cv2.bitwise_and(img, img, mask=mask)
plt.imshow(cv2.cvtColor(pcb_extracted, cv2.COLOR_BGR2RGB))
plt.title("Extracted Motherboard Image")
plt.axis("off")
plt.show()

In [ ]:
# STEP 2: YOLOV11 TRAINING

# Extract data zip folder that is inside Project 3 Data zip folder
data_zip = os.path.join(extract_path, 'Project 3 Data', 'data.zip')
data_dir = '/content/project3_data/data'
with zipfile.ZipFile(data_zip, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

# Path to data.yaml
data_yaml = '/content/project3_data/data/data/data.yaml'

# YOLOv11 Nano pretrained model
model = YOLO("yolo11n.pt")

# Fine-tune model
history = model.train(
    data=data_yaml,
    epochs=100,       # <200
    batch=32,
    imgsz=1200,      # ≥900
    name="mdl"
)

In [ ]:
# STEP 3: EVALUATION

# Get all images to be evaluated
evaluation_images = [
    "/content/project3_data/data/data/evaluation/arduno.jpg",
    "/content/project3_data/data/data/evaluation/ardmega.jpg",
    "/content/project3_data/data/data/evaluation/rasppi.jpg"
]

# Loop over each evaluation image and show predictions with bounding boxes
for img_file in evaluation_images:
    result = model.predict(img_file, conf=0.5)[0]  # take the first (and only) result
    result.show()
